In [5]:
from typing import List


class Node:

    def __init__(self):
        self.val = 0
        self.left = None
        self.right = None
        self.lazy = 0


class NumArray:

    def __init__(self, nums: List[int]):
        self.root = Node()
        self.lgh = len(nums)
        for i in range(self.lgh):
            self.updateTree(self.root, 0, self.lgh - 1, i, i, nums[i])

    def update(self, index: int, val: int) -> None:
        self.updateTree(self.root, 0, self.lgh - 1, index, index, val)

    def sumRange(self, left: int, right: int) -> int:
        return self.queryTree(self.root, 0, self.lgh - 1, left, right)

    def pushUp(self, node: Node):
        node.val = node.left.val + node.right.val

    def pushDown(self, node: Node):  # 下推node的lazy标记
        if not node.left: node.left = Node()
        if not node.right: node.right = Node()
        if node.lazy == 0: return
        node.left.val = node.lazy
        node.left.lazy = node.lazy
        node.right.val = node.lazy
        node.right.lazy = node.lazy
        node.lazy = 0

    def updateTree(self, node: Node, si, ei, l, r, val):  # 某个根节点代表范围[si,ei] 更新[l,r]为val
        if l <= si and ei <= r:
            node.val = val
            node.lazy = val
            return
        mi = si + ei >> 1
        self.pushDown(node)
        if l <= mi:
            self.updateTree(node.left, si, mi, l, r, val)
        if r >= mi + 1:
            self.updateTree(node.right, mi + 1, ei, l, r, val)
        self.pushUp(node)

    def queryTree(self, node: Node, si, ei, l, r):  # 某个根节点代表范围[si,ei] 查询[l,r]值
        if l <= si and ei <= r:  # 开头直接返回情况[si,ei]=[l,r] 其他递归返回情况 [2,4]=[2,2]+[3,4]
            return node.val
        mi = si + ei >> 1
        ans = 0
        self.pushDown(node)
        if l <= mi:  # 左节点有交集
            ans += self.queryTree(node.left, si, mi, l, r)
        if mi + 1 <= r:  # 右节点有交集
            ans += self.queryTree(node.right, mi + 1, ei, l, r)
        return ans


# Your NumArray object will be instantiated and called as such:
obj = NumArray([9,-8])
obj.update(0,3)
obj.sumRange(1,1)

-8

In [2]:
from typing import List
class Node:

    def __init__(self):
        self.val = 0
        self.left = None
        self.right = None
        self.lazy = None # lazy代表该节点已经改变 但子节点未下传改变


class SegmentTree:

    def __init__(self):
        self.root = Node()

    def update(self, node, si, ei, l, r, val):
        '''
        节点node范围[si,ei]
        更新[l,r]所有值更新为val
        '''
        if ei < l or si > r:
            return
        if l <= si <= ei <= r:
            node.val = val
            node.lazy = val
            return
        mi = si + ei >> 1  # 对应左树范围 [si,mi] 右树范围 [mi+1,ei]
        self.pushDown(node)
        if mi >= l:
            self.update(node.left, si, mi, l, r, val)
        if mi + 1 <= r:
            self.update(node.right, mi + 1, ei, l, r, val)
        node.val = node.left.val + node.right.val

    def query(self, node, si, ei, l, r):
        '''
        节点node范围[si,ei]
        查询[l,r]所有值的和
        '''
        if ei < l or si > r:
            return 0
        if l <= si <= ei <= r: 
            return node.val
        mi = si + ei >> 1
        self.pushDown(node)
        ans = 0
        if mi >= l:
            ans += self.query(node.left, si, mi, l, r)
        if mi + 1 <= r:
            ans += self.query(node.right, mi + 1, ei, l, r)
        return ans

    def pushDown(self, node):
        if not node.left:
            node.left = Node()
        if not node.right:
            node.right = Node()
        if node.lazy is None:
            return
        node.left.val = node.lazy
        node.right.val = node.lazy
        node.left.lazy = node.lazy
        node.right.lazy = node.lazy
        node.lazy = None


class NumArray:

    def __init__(self, nums: List[int]):
        self.st = SegmentTree()
        self.lgh = len(nums)
        for i in range(len(nums)):
            self.st.update(self.st.root, 0, self.lgh - 1, i, i, nums[i])

    def update(self, index: int, val: int) -> None:
        self.st.update(self.st.root, 0, self.lgh - 1, index, index, val)

    def sumRange(self, left: int, right: int) -> int:
        return self.st.query(self.st.root, 0, self.lgh - 1, left, right)


In [3]:
from typing import List


class Node:

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.lgh = ei - si + 1
        self.val = 0  # 代表[si,ei]区间和
        self.left = None
        self.right = None
        self.lazy = None  # 父区间已更新 但未下传


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def push_down(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy is None:
            return
        node.left.val = node.left.lgh * node.lazy
        node.right.val = node.right.lgh * node.lazy
        node.left.lazy = node.lazy
        node.right.lazy = node.lazy
        node.lazy = None

    def update(self, node: Node, l, r, val):  # node节点中[l,r]每个值设为val
        if node.ei < l or node.si > r:
            return
        if l <= node.si <= node.ei <= r:
            node.val = node.lgh * val
            node.lazy = val
            return
        self.push_down(node)
        if node.left.ei >= l:
            self.update(node.left, l, r, val)
        if node.right.si <= r:
            self.update(node.right, l, r, val)
        node.val = node.left.val + node.right.val

    def query(self, node: Node, l, r):  # 查询node结点中[l,r]的区间和
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si <= node.ei <= r:
            return node.val
        self.push_down(node)
        ans = 0
        if node.left.ei >= l:
            ans += self.query(node.left, l, r)
        if node.right.si <= r:
            ans += self.query(node.right, l, r)
        return ans


class NumArray:

    def __init__(self, nums: List[int]):
        self.st = SegmentTree(0, len(nums) - 1)
        for i, n in enumerate(nums):
            self.st.update(self.st.root, i, i, n)

    def update(self, index: int, val: int) -> None:
        self.st.update(self.st.root, index, index, val)

    def sumRange(self, left: int, right: int) -> int:
        return self.st.query(self.st.root, left, right)


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(index,val)
# param_2 = obj.sumRange(left,right)

In [4]:
3*10**4-1

29999